In [3]:
import json
import time
from datetime import datetime, timedelta
import random
import os

class MixingSimulation:
    def __init__(self):
        self.Volume = 200  # Total volume in Litres
        self.AM_ratio = 0.495
        self.PVDF_ratio = 0.05
        self.CB_ratio = 0.045
        self.NMP_ratio = 0.41

        # Density of each component (g/cm³)
        self.RHO_AM = 2.26
        self.RHO_CB = 1.8
        self.RHO_PVDF = 1.78
        self.RHO_NMP = 1

        # Empirical values of each component
        self.a = 0.85  # AM
        self.b = 2.2  # Binder
        self.c = 0.3  # CB
        self.s = -0.4 # Solvent

        # Initial component volumes
        self.NMP = self.NMP_ratio * self.Volume
        self.PVDF = 0
        self.CB = 0
        self.AM = 0

        self.results = []
        self.total_time = 0
        self.start_datetime = datetime.now()
        

    def run(self, step_percent=0.02, pause_sec=0.1):
        self._add_in_steps('PVDF', self.PVDF_ratio, step_percent, pause_sec, "PVDF")
        self._add_in_steps('CB', self.CB_ratio, step_percent, pause_sec, "Carbon Black")
        self._add_in_steps('AM', self.AM_ratio, step_percent, pause_sec, "Active Material")

    def _add_in_steps(self, component, ratio, step_percent, pause_sec, label):
        total_volume_to_add = ratio * self.Volume
        step_volume = step_percent * total_volume_to_add
        num_steps = int(1 / step_percent)

        last_saved_time = time.time()

        for _ in range(num_steps):
            self.total_time == 0
            setattr(self, component, getattr(self, component) + step_volume)

            timestamp = (self.start_datetime + timedelta(seconds=self.total_time)).isoformat()
            temperature = round(random.uniform(20, 25), 2)
            pressure = round(random.uniform(1, 2), 2)
            rpm = random.randint(300, 600)
            density = self._calculate_density()
            viscosity = self._calculate_viscosity()
            yield_stress = self._calculate_yield_stress()

            result = {
                "TimeStamp": timestamp,
                "Duration": self.total_time,
                "AM": round(self.AM, 3),
                "CB": round(self.CB, 3),
                "PVDF": round(self.PVDF, 3),
                "NMP": round(self.NMP, 3),
                "Temperature": temperature,
                "Pressure": pressure,
                "Speed (RPM)": rpm,
                "Density": round(density, 4),
                "Viscosity": round(viscosity, 2),
                "Yield Stress": round(yield_stress, 2)
            }
            self.results.append(result)

            now = time.time()
            if now - last_saved_time >= 0.1:
                os.makedirs("simulation_output", exist_ok=True)
                filename = f"simulation_output/result_at_{int(self.total_time)}s.json"
                try:
                    with open(filename, "w") as f:
                        json.dump(result, f, indent=4)
                except Exception as e:
                    print(f"Error writing result to file: {e}")
                last_saved_time = now
            self.total_time += 5
            time.sleep(pause_sec)

    def _calculate_density(self):
        m_NMP = self.NMP * self.RHO_NMP
        m_PVDF = self.PVDF * self.RHO_PVDF
        m_CB = self.CB * self.RHO_CB
        m_AM = self.AM * self.RHO_AM

        total_mass = m_NMP + m_PVDF + m_CB + m_AM
        total_volume = self.NMP + self.PVDF + self.CB + self.AM
        return total_mass / total_volume if total_volume > 0 else 0

    def _calculate_viscosity(self, max_solid_fraction=0.63, intrinsic_viscosity=3):
        total_volume = self.NMP + self.PVDF + self.CB + self.AM
        if total_volume == 0:
            return 0

        solid_volume = self.PVDF + self.CB + self.AM
        phi = solid_volume / total_volume
        if phi >= max_solid_fraction:
            phi = max_solid_fraction - 0.001

        try:
            viscosity = (1 - (phi / max_solid_fraction)) ** (-intrinsic_viscosity * max_solid_fraction)
            return viscosity * 0.017
        except Exception as e:
            print(f"Viscosity calculation error: {e}")
            return float('inf')

    def _calculate_yield_stress(self):
        m_NMP = self.NMP * self.RHO_NMP
        m_PVDF = self.PVDF * self.RHO_PVDF
        m_CB = self.CB * self.RHO_CB
        m_AM = self.AM * self.RHO_AM

        y_AM = self.a * m_AM
        y_PVDF = self.b * m_PVDF
        y_CB = self.c * m_CB
        y_NMP = self.s * m_NMP
        return y_NMP + y_PVDF + y_CB + y_AM

    def save_to_json(self, filename="simulation_output/final_results.json"):
        os.makedirs("simulation_output", exist_ok=True)
        try:
            with open(filename, "w") as f:
                json.dump(self.results, f, indent=4)
        except Exception as e:
            print(f"Failed to save final results: {e}")


if __name__ == "__main__":
    sim = MixingSimulation()
    sim.run()
    sim.save_to_json()

    in_viscosity = sim.results[-1]["Viscosity"]
    solidContent = (sim.results[-1]["AM"] + sim.results[-1]["CB"] + sim.results[-1]["PVDF"]) / (sim.results[-1]["NMP"] + sim.results[-1]["AM"] + sim.results[-1]["CB"] + sim.results[-1]["PVDF"]) 
    mSolvent = sim.results[-1]["NMP"]
    print(in_viscosity)
    print(solidContent)

3.11
0.59


    Coating Process

In [ ]:
#########COATING---PROCESS##########
import numpy as np
import random
import time
import numpy as np
import json
from datetime import datetime, timedelta

class CoatingSimulation:
    def __init__(self):
        # Variables from Mixing
        self.viscosity_pa = in_viscosity
        self.solid_content = solidContent  # fraction (e.g., 0.55 for 55%)

        # Parameters & possible Input
        self.coating_speed = 0.05  # m/s (0,05 - 5 m/s)
        self.gap_height = 200e-6  # meters (50e-6 to 300 e-6)
        self.flow_rate = 5e-6  # m³/s (Possibly fixed)
        self.coating_width = 0.5  # m (possibly fixed)
        self.K = 100  # Defect risk constant (10-100)
        self.base_std = 0.01  # Base standard deviation
        self.nominal_shear_rate = 500  # 1/s
        self.sample_points = 800

        # Time tracking
        self.start_time = datetime.now()
        self.total_time = 0

        # For storing output
        self.results = []

    def simulate(self, end_time=100, save_json=False, filename="coating_simulation.json"):
        """Simulate the coating process and log results."""
        for t in range(0, end_time + 1, 5):
            self.total_time = t
            shear_rate = self.calculate_shear_rate()
            viscosity = self.viscosity_pa
            wet_thickness = self.calculate_wet_thickness()
            dry_thickness = self.calculate_dry_thickness(wet_thickness)
            defect_risk = self.check_defect_risk(shear_rate, viscosity)
            uniformity_std = self.calculate_uniformity(shear_rate)

            result = {
                "timestamp": str(self.start_time + timedelta(seconds=t)),
                "duration": t,
                "shear_rate_1_per_s": round(shear_rate, 4),
                "viscosity_Pa_s": round(viscosity, 4),
                "wet_thickness_m": round(wet_thickness, 5),
                "dry_thickness_m": round(dry_thickness, 5),
                "defect_risk": defect_risk,
                "uniformity_std": round(uniformity_std, 4),
            }

            self.results.append(result)

            # Print to console (optional)
            print(json.dumps(result, indent=2))

            time.sleep(0.1)  # Simulated time delay

        if save_json:
            with open(filename, "w") as f:
                json.dump(self.results, f, indent=2)

    def calculate_shear_rate(self):
        return self.coating_speed / self.gap_height

    def calculate_wet_thickness(self):
        return self.flow_rate / (self.coating_speed * self.coating_width)

    def calculate_dry_thickness(self, wet_thickness):
        return wet_thickness * self.solid_content

    def check_defect_risk(self, shear_rate, viscosity):
        return (self.coating_speed / self.gap_height) > (self.K * viscosity)

    def calculate_uniformity(self, shear_rate):
        return self.base_std * (shear_rate / self.nominal_shear_rate)


# Run the simulation with example values
if __name__ == "__main__":
    sim = CoatingSimulation()
    sim.simulate(end_time=100, save_json=True)


    wet_thickness_coating = sim.results[-1]["wet_thickness_m"]


{
  "timestamp": "2025-05-13 01:19:47.391690",
  "duration": 0,
  "shear_rate_1_per_s": 250.0,
  "viscosity_Pa_s": 3.11,
  "wet_thickness_m": 0.0002,
  "dry_thickness_m": 0.00012,
  "defect_risk": false,
  "uniformity_std": 0.005
}
{
  "timestamp": "2025-05-13 01:19:52.391690",
  "duration": 5,
  "shear_rate_1_per_s": 250.0,
  "viscosity_Pa_s": 3.11,
  "wet_thickness_m": 0.0002,
  "dry_thickness_m": 0.00012,
  "defect_risk": false,
  "uniformity_std": 0.005
}
{
  "timestamp": "2025-05-13 01:19:57.391690",
  "duration": 10,
  "shear_rate_1_per_s": 250.0,
  "viscosity_Pa_s": 3.11,
  "wet_thickness_m": 0.0002,
  "dry_thickness_m": 0.00012,
  "defect_risk": false,
  "uniformity_std": 0.005
}
{
  "timestamp": "2025-05-13 01:20:02.391690",
  "duration": 15,
  "shear_rate_1_per_s": 250.0,
  "viscosity_Pa_s": 3.11,
  "wet_thickness_m": 0.0002,
  "dry_thickness_m": 0.00012,
  "defect_risk": false,
  "uniformity_std": 0.005
}
{
  "timestamp": "2025-05-13 01:20:07.391690",
  "duration": 20,
  "sh

In [ ]:
##########Drying Process#########

class DryingSimulation:
    def __init__(self, wet_thickness, solid_content, web_speed):
        # Inputs from Coating
        self.wet_thickness = wet_thickness                # m
        self.solid_content = solid_content        # fraction (e.g., 0.5)
        self.web_speed = web_speed                # m/s

        # Fixed Parameters
        self.coating_width = 0.5    # m
        self.h_air = 0.1            # m
        self.drying_length = 10     # m
        self.T_dry = 100            # °C
        self.V_air = 1.0            # m³/s
        self.H_air = 30             # % RH
        self.density = 1500         # slurry density (kg/m³)
        self.solvent_density = 800  # kg/m³
        self.k0 = 0.001             # base mass transfer coefficient (m/s)
        self.delta_t = 1            # time step (s)
        self.max_safe_evap_rate = 0.001           # kg/m²/s

        # Derived Constants
        self.mass_transfer_coeff = self.k0 * (self.V_air / (self.coating_width * self.h_air))
        self.C_surface = 1.0
        self.C_air = self.H_air / 100
        self.delta_dry = self.delta_wet * self.solid_content
        self.M_solvent = self.delta_wet * (1 - self.solid_content) * self.density  # kg/m²
        self.area = self.coating_width * 1  # Assume 1m length
        self.residence_time = self.drying_length / self.web_speed
        self.time_steps = int(self.residence_time / self.delta_t)

        # Time and Output
        self.total_time = 0
        self.start_time = datetime.now()
        self.results = []

    def simulate(self, save_json=False, filename="drying_output/drying_results.json"):
        os.makedirs("drying_output", exist_ok=True)

        for t in range(0, self.time_steps + 1, self.delta_t):
            self.total_time = t

            # Evaporation rate (kg/s)
            evap_rate = -self.mass_transfer_coeff * self.area * (self.C_surface - self.C_air)

            # Update solvent mass
            self.M_solvent -= (evap_rate / self.area) * self.delta_t
            if self.M_solvent < 0:
                self.M_solvent = 0

            # Update coating thickness
            delta_coat = self.delta_dry + (self.M_solvent / self.solvent_density)

            # Check defect risk
            defect_risk = abs(evap_rate / self.area) > self.max_safe_evap_rate

            result = {
                "timestamp": str(self.start_time + timedelta(seconds=t)),
                "duration": t,
                "wet_thickness_m": round(self.delta_wet, 6),
                "dry_thickness_m": round(self.delta_dry, 6),
                "M_solvent_kg_m2": round(self.M_solvent, 6),
                "evaporation_rate_kg_s": round(evap_rate, 6),
                "coating_thickness_m": round(delta_coat, 6),
                "defect_risk": defect_risk
            }

            self.results.append(result)
            print(json.dumps(result, indent=2))

            if self.M_solvent == 0:
                break

            time.sleep(0.05)

        if save_json:
            with open(filename, "w") as f:
                json.dump(self.results, f, indent=2)


# Run the simulation with example values
if __name__ == "__main__":
    drying_sim = DryingSimulation(
        delta_wet=0.00012,        # 120 from coating
        solid_content=0.5,        # 50 from mixing
        web_speed=0.1             # m/s from coating
    )
    drying_sim.simulate(save_json=True)

TypeError: DryingSimulation.__init__() got an unexpected keyword argument 'delta_wet'

In [ ]:
####Calendaring####

class CalendaringProcess:
    def __init__(self):
        # Input from drying stage
        self.delta_dry = 80e-6         # Dry coating thickness (m) ##Drying Process
        self.phi_initial = 0.4         # Initial porosity 
        
        ###Controlled Variables
        self.E = 500e6                 # Elastic modulus (Pa)
        self.k_p = 3.0                 # Porosity reduction constant
        
        ###User Input###
        self.h_roll = 60e-6            # Roll gap (m) ## 50e-6 - 500e-6
        self.P_roll = 50e6             # Applied roll pressure (Pa) ## 20e6, 50e6
        self.v_roll = 1.0              # Roll speed (m/s) ## 0.5 to 2
        
        # Environment
        self.T = 25                    # Temperature (°C) ##Fixed

        # Time
        self.delta_t = 1               # Time step (s)
        self.total_time = 60           # Total simulation time (s)

        self.start_time = datetime.now()
        self.results = []

    def epsilon(self):
        return (self.delta_dry - self.h_roll) / self.delta_dry

    def sigmaCalc(self, epsilon_val):
        return self.E * epsilon_val

    def porosityReduction(self, epsilon_val):
        return self.phi_initial * np.exp(-self.k_p * epsilon_val)

    def defectRisk(self, applied_sigma, calculated_sigma, time_step):
        if applied_sigma > 2 * calculated_sigma:
            msg = f"Warning: Excessive pressure at t = {time_step}s may cause cracks."
            return True, msg
        else:
            return False, ""

    def simulation(self, save_json=False, filename="calendaring_simulation.json"):
        # Initial calculations
        epsilon_val = self.epsilon()
        sigma_val = self.P_roll
        sigma_theory = self.sigmaCalc(epsilon_val)
        porosity = self.porosityReduction(epsilon_val)
        final_thick = self.h_roll

        # Simulation loop
        for t in range(0, self.total_time + 1, self.delta_t):
            # Update roll gap, speed, and pressure dynamically
            self.h_roll = 60e-6 * (1 - t / self.total_time)  # Decrease roll gap over time
            self.v_roll = max(0.5, 1.0 - t / (self.total_time * 2))  # Gradually reduce speed
            self.P_roll = 50e6 * (1 - self.h_roll / self.delta_dry)  # Increase pressure as roll gap decreases

            # Recalculate strain (epsilon) based on the updated roll gap
            epsilon_val = self.epsilon()
            sigma_theory = self.sigmaCalc(epsilon_val)
            porosity = self.porosityReduction(epsilon_val)
            final_thick = self.h_roll

            # Check for defect risk
            risk, warning_msg = self.defectRisk(sigma_val, sigma_theory, t)

            # Record results
            timestamp = self.start_time + timedelta(seconds=t)
            result = {
                "timestamp": str(timestamp),
                "time_elapsed_s": t,
                "gap": self.h_roll,
                "speed": self.v_roll,
                "pressure": self.P_roll,
                "strain_epsilon": round(epsilon_val, 6),
                "applied_pressure_Pa": sigma_val,
                "calculated_stress_Pa": round(sigma_theory, 2),
                "final_thickness_m": round(final_thick, 8),
                "porosity_after_calendaring": round(porosity, 4),
                "defect_risk": risk,
            }

            if warning_msg:
                result["warning"] = warning_msg

            self.results.append(result)
            print(json.dumps(result, indent=2))

            # Simulate real-time delay
            time.sleep(0.1)

        # Save results to JSON file if requested
        if save_json:
            with open(filename, "w") as f:
                json.dump(self.results, f, indent=2)
            print(f"\nSimulation results saved to {filename}")

# Run the simulation
if __name__ == "__main__":
    cal = CalendaringProcess()
    cal.simulation(save_json=True)


{
  "timestamp": "2025-05-13 01:14:55.046705",
  "time_elapsed_s": 0,
  "gap": 6e-05,
  "speed": 1.0,
  "pressure": 12500000.0,
  "strain_epsilon": 0.25,
  "applied_pressure_Pa": 50000000.0,
  "calculated_stress_Pa": 125000000.0,
  "final_thickness_m": 6e-05,
  "porosity_after_calendaring": 0.1889,
  "defect_risk": false
}
{
  "timestamp": "2025-05-13 01:14:56.046705",
  "time_elapsed_s": 1,
  "gap": 5.9e-05,
  "speed": 0.9916666666666667,
  "pressure": 13125000.000000004,
  "strain_epsilon": 0.2625,
  "applied_pressure_Pa": 50000000.0,
  "calculated_stress_Pa": 131250000.0,
  "final_thickness_m": 5.9e-05,
  "porosity_after_calendaring": 0.182,
  "defect_risk": false
}
{
  "timestamp": "2025-05-13 01:14:57.046705",
  "time_elapsed_s": 2,
  "gap": 5.8e-05,
  "speed": 0.9833333333333333,
  "pressure": 13750000.000000002,
  "strain_epsilon": 0.275,
  "applied_pressure_Pa": 50000000.0,
  "calculated_stress_Pa": 137500000.0,
  "final_thickness_m": 5.8e-05,
  "porosity_after_calendaring": 0.